In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random 
from sklearn.metrics import classification_report
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [4]:
data=pd.read_csv("/content/nerupd.csv",encoding="latin1",delimiter='\t')
data.head()

,Sentence,Tag
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [6]:
data.shape

(47949, 2)

In [7]:
data.head()

,Sentence,Tag
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [8]:
agg_data = data.copy(True)
agg_data['tokenised_sentences']=agg_data['Sentence'].apply(lambda x:x.split())
agg_data['tag_list']=agg_data['Tag'].apply(lambda x:x.split())
agg_data.head()

,Sentence,Tag,tokenised_sentences,tag_list
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O..."
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]"
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...,"[U.N., relief, coordinator, Jan, Egeland, said...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo..."


In [9]:
agg_data['len_sentence']=agg_data['tokenised_sentences'].apply(lambda x:len(x))
agg_data['len_tag']=agg_data['tag_list'].apply(lambda x:len(x))
agg_data['is_equal']=agg_data.apply(lambda row:1 if row['len_sentence']==row['len_tag'] else 0,axis=1)
agg_data['is_equal'].value_counts()

1    47836
0      113
Name: is_equal, dtype: int64

In [10]:
agg_data.head()

,Sentence,Tag,tokenised_sentences,tag_list,len_sentence,len_tag,is_equal
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...",24,24,1
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...",25,25,1
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...",32,32,1
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]",11,11,1
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...,"[U.N., relief, coordinator, Jan, Egeland, said...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...",35,35,1


In [11]:
agg_data[agg_data['is_equal']==0].head()

,Sentence,Tag,tokenised_sentences,tag_list,len_sentence,len_tag,is_equal
1463,"You do me a grave injustice , said the Boss ...",O O O O O O O O O O O O O O O O O O O O,"[You, do, me, a, grave, injustice, ,, said, th...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18,20,0
1464,I came to Canada solely because of its politi...,O O O O B-geo O O O O O O O O O O O O O O O O O O,"[I, came, to, Canada, solely, because, of, its...","[O, O, O, O, B-geo, O, O, O, O, O, O, O, O, O,...",21,23,0
1465,"Pray forgive me , said the Citizen of Montre...",O O O O O O O O O O B-org O,"[Pray, forgive, me, ,, said, the, Citizen, of,...","[O, O, O, O, O, O, O, O, O, O, B-org, O]",10,12,0
2220,"Not so fast , friend , said the Hunter .",O O O O O O O O O O O O,"[Not, so, fast, ,, friend, ,, said, the, Hunte...","[O, O, O, O, O, O, O, O, O, O, O, O]",10,12,0
2221,"I have now got you under bit and spur , and p...",O O O O O O O O O O O O O O O O O O O O O O O,"[I, have, now, got, you, under, bit, and, spur...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",21,23,0


In [12]:
agg_data=agg_data[agg_data['is_equal']!=0]

In [13]:
agg_data.shape

(47836, 7)

In [14]:
sentences_list=agg_data['Sentence'].tolist()
tags_list=agg_data['tag_list'].tolist()

print("Number of Sentences in the Data ",len(sentences_list))
print("Are number of Sentences and Tag list equal ",len(sentences_list)==len(tags_list))

Number of Sentences in the Data  47836
Are number of Sentences and Tag list equal  True


In [16]:
sentences_list[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [17]:
tags_list[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [19]:
tokeniser= tf.keras.preprocessing.text.Tokenizer(lower=False,filters='')

tokeniser.fit_on_texts(sentences_list)

In [20]:
print("Vocab size of Tokeniser ",len(tokeniser.word_index)+1) ## Adding one since 0 is reserved for padding

Vocab size of Tokeniser  35089


In [21]:
tokeniser.index_word[326]

'West'

In [22]:
encoded_sentence=tokeniser.texts_to_sequences(sentences_list)
print("First Original Sentence ",sentences_list[0])
print("First Encoded Sentence ",encoded_sentence[0])
print("Is Length of Original Sentence Same as Encoded Sentence ",len(sentences_list[0].split())==len(encoded_sentence[0]))
print("Length of First Sentence ",len(encoded_sentence[0]))

First Original Sentence  Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
First Encoded Sentence  [1114, 4, 1159, 16, 1848, 229, 477, 6, 532, 1, 155, 5, 58, 8, 580, 1, 848, 4, 179, 87, 21, 15, 52, 2]
Is Length of Original Sentence Same as Encoded Sentence  True
Length of First Sentence  24


In [26]:
print(len(list(set(data['Tag'].values))))

33316


In [28]:
tagset = set()
for x in data['Tag']:
    for y in x.split():
        tagset.add(y)
print(len(list(tagset)))

17


In [29]:
tags=list(tagset)
#print(tags)
num_tags=len(tags)
print("Number of Tags ",num_tags)

tags_map={tag:i for i,tag in enumerate(tags)}
print("Tags Map ",tags_map)

Number of Tags  17
Tags Map  {'B-org': 0, 'B-per': 1, 'I-per': 2, 'I-tim': 3, 'I-nat': 4, 'O': 5, 'B-art': 6, 'I-eve': 7, 'I-art': 8, 'B-geo': 9, 'I-org': 10, 'I-gpe': 11, 'B-eve': 12, 'B-gpe': 13, 'B-tim': 14, 'B-nat': 15, 'I-geo': 16}


In [30]:
reverse_tag_map={v: k for k, v in tags_map.items()}

In [31]:
print("Reverse Tags Map ", reverse_tag_map)

Reverse Tags Map  {0: 'B-org', 1: 'B-per', 2: 'I-per', 3: 'I-tim', 4: 'I-nat', 5: 'O', 6: 'B-art', 7: 'I-eve', 8: 'I-art', 9: 'B-geo', 10: 'I-org', 11: 'I-gpe', 12: 'B-eve', 13: 'B-gpe', 14: 'B-tim', 15: 'B-nat', 16: 'I-geo'}


In [32]:
encoded_tags=[[tags_map[w] for w in tag] for tag in tags_list]
print("First Sentence ",sentences_list[0])
print('First Sentence Original Tags ',tags_list[0])
print("First Sentence Encoded Tags ",encoded_tags[0])
print("Is length of Original Tags and Encoded Tags same ",len(tags_list[0])==len(encoded_tags[0]))
print("Length of Tags for First Sentence ",len(encoded_tags[0]))

First Sentence  Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
First Sentence Original Tags  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']
First Sentence Encoded Tags  [5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 13, 5, 5, 5, 5, 5]
Is length of Original Tags and Encoded Tags same  True
Length of Tags for First Sentence  24


In [33]:
max_sentence_length=max([len(s.split()) for s in sentences_list])
print(max_sentence_length)

104


In [34]:
max_len=128
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

padded_encoded_sentences=pad_sequences(maxlen=max_len,sequences=encoded_sentence,padding="post",value=0)
padded_encoded_tags=pad_sequences(maxlen=max_len,sequences=encoded_tags,padding="post",value=tags_map['O'])

print("Shape of Encoded Sentence ",padded_encoded_sentences.shape)
print("Shape of Encoded Labels ",padded_encoded_tags.shape)

print("First Encoded Sentence Without Padding ",encoded_sentence[0])
print("First Encoded Sentence with padding ",padded_encoded_sentences[0])
print("First Sentence Encoded Label without Padding ",encoded_tags[0])
print("First Sentence Encoded Label with Padding ",padded_encoded_tags[0])

Shape of Encoded Sentence  (47836, 128)
Shape of Encoded Labels  (47836, 128)
First Encoded Sentence Without Padding  [1114, 4, 1159, 16, 1848, 229, 477, 6, 532, 1, 155, 5, 58, 8, 580, 1, 848, 4, 179, 87, 21, 15, 52, 2]
First Encoded Sentence with padding  [1114    4 1159   16 1848  229  477    6  532    1  155    5   58    8
  580    1  848    4  179   87   21   15   52    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
First Sentence Encoded Label without Padding  [5, 5, 5, 5, 5, 5, 9, 5, 5, 5, 5, 5, 9, 5, 5, 

In [35]:
target= [to_categorical(i,num_classes = num_tags) for i in  padded_encoded_tags]
print("Shape of Labels  after converting to Categorical for first sentence ",target[0].shape)

Shape of Labels  after converting to Categorical for first sentence  (128, 17)


In [36]:
target[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_val_test,y_train,y_val_test = train_test_split(padded_encoded_sentences,target,test_size = 0.3,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_val_test,y_val_test,test_size = 0.2,random_state=42)
print("Input Train Data Shape ",X_train.shape)
print("Train Labels Length ",len(y_train))
print("Input Test Data Shape ",X_test.shape)
print("Test Labels Length ",len(y_test))

print("Input Validation Data Shape ",X_val.shape)
print("Validation Labels Length ",len(y_val))

Input Train Data Shape  (33485, 128)
Train Labels Length  33485
Input Test Data Shape  (2871, 128)
Test Labels Length  2871
Input Validation Data Shape  (11480, 128)
Validation Labels Length  11480


In [38]:
print("First Sentence in Training Data ",X_train[0])
print("First sentence Label ",y_train[0])
print("Shape of First Sentence -Train",X_train[0].shape)
print("Shape of First Sentence Label  -Train",y_train[0].shape)

First Sentence in Training Data  [  59    7  156   80    3    1 1447 1983   20  116 1103    1 2251   60
   65   44  306  106  281  370    4    1  218  522    2    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
First sentence Label  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of First Sentence -Train (128,)
Shape of First Sentence Label  -Train (128, 17)


In [39]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional

In [68]:
embedding_dim=128
vocab_size=len(tokeniser.word_index)+1
lstm_units=128
max_len=128

input_word = Input(shape = (max_len,))
model = Embedding(input_dim = vocab_size+1,output_dim = embedding_dim,input_length = max_len)(input_word)
model = Bidirectional(LSTM(units=embedding_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat')(model)
model = LSTM(units=embedding_dim,return_sequences=True)(model)
out = TimeDistributed(Dense(num_tags,activation = 'softmax'))(model)
model = Model(input_word,out)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 128, 128)          4491520   
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 256)          263168    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128, 128)          197120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 128, 17)           2193      
Total params: 4,954,001
Trainable params: 4,954,001
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [70]:
history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size = 32,epochs = 1)

1047/1047 [==============================] - 1962s 2s/step - loss: 0.0801 - accuracy: 0.9818 - val_loss: 0.0308 - val_accuracy: 0.9914


In [71]:
preds=model.predict(X_test) ## Predict using model on Test Data

In [72]:
def evaluatePredictions(test_data,preds,actual_preds):
    print("Shape of Test Data Array",test_data.shape)
    y_actual=np.argmax(np.array(actual_preds),axis=2)
    y_pred=np.argmax(preds,axis=2)
    num_test_data=test_data.shape[0]
    print("Number of Test Data Points ",num_test_data)
    data=pd.DataFrame()
    df_list=[]
    for i in range(num_test_data):
        test_str=list(test_data[i])
        df=pd.DataFrame()
        df['test_input']=test_data[i]
        df['test_tokens']=test_str
        df['tokens']=df['test_tokens'].apply(lambda x:tokeniser.index_word[x] if x!=0 else '<PAD>')
        df['actual_target_index']=list(y_actual[i])
        df['pred_target_index']=list(y_pred[i])
        df['actual_target_tag']=df['actual_target_index'].apply(lambda x:reverse_tag_map[x])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tag_map[x])
        df['id']=i+1
        df_list.append(df)
    data=pd.concat(df_list)
    pred_data=data[data['tokens']!='<PAD>']
    accuracy=pred_data[pred_data['actual_target_tag']==pred_data['pred_target_tag']].shape[0]/pred_data.shape[0]
    
    return pred_data,accuracy

In [73]:
pred_data,accuracy=evaluatePredictions(X_test,preds,y_test)

Shape of Test Data Array (2871, 128)
Number of Test Data Points  2871


In [74]:
y_pred=pred_data['pred_target_tag'].tolist()
y_actual=pred_data['actual_target_tag'].tolist()

In [75]:
print(classification_report(y_actual,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        23
       B-eve       0.00      0.00      0.00        19
       B-geo       0.82      0.81      0.82      2352
       B-gpe       0.92      0.85      0.88      1011
       B-nat       0.00      0.00      0.00        10
       B-org       0.57      0.64      0.61      1260
       B-per       0.74      0.62      0.68      1060
       B-tim       0.89      0.73      0.80      1184
       I-art       0.00      0.00      0.00        19
       I-eve       0.00      0.00      0.00        23
       I-geo       0.77      0.67      0.72       456
       I-gpe       0.00      0.00      0.00        10
       I-nat       0.00      0.00      0.00         4
       I-org       0.64      0.67      0.66       984
       I-per       0.82      0.76      0.79      1085
       I-tim       0.95      0.35      0.51       449
           O       0.98      0.99      0.99     53115

    accuracy              

In [76]:
pred_data[pred_data['actual_target_tag']=="B-art"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
5,19245,19245,Tree,6,5,B-art,O,238
16,3231,3231,Emergency,6,5,B-art,O,334
9,31327,31327,Rolls,6,5,B-art,O,391
26,2797,2797,English,6,13,B-art,B-gpe,417
6,296,296,French,6,13,B-art,B-gpe,612
8,3187,3187,al-Jazeera,6,0,B-art,B-org,1064
9,5647,5647,Facebook,6,0,B-art,B-org,1181
1,128,128,U.N.,6,0,B-art,B-org,1257
6,3887,3887,WFP,6,0,B-art,B-org,1257
21,1013,1013,Georgia,6,9,B-art,B-geo,1346


In [77]:
pred_data[pred_data['actual_target_tag']=="B-nat"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
6,948,948,Katrina,15,5,B-nat,O,200
6,4006,4006,Rita,15,0,B-nat,B-org,290
18,10306,10306,H5N2,15,5,B-nat,O,459
14,709,709,2007,15,5,B-nat,O,510
21,24632,24632,Euphausia,15,5,B-nat,O,595
34,24635,24635,Dissostichus,15,5,B-nat,O,595
14,824,824,Hurricane,15,5,B-nat,O,1106
8,948,948,Katrina,15,5,B-nat,O,1203
37,948,948,Katrina,15,5,B-nat,O,1350
0,948,948,Katrina,15,5,B-nat,O,2437


In [78]:
y_pred[0]

'B-gpe'

In [79]:
y_actual[0]

'B-gpe'

In [80]:
pred_data['tokens'][1]

1     officials
1           and
1          said
1     spokesman
1      National
        ...    
1       hundred
1    Ambassador
1           was
1            of
1          week
Name: tokens, Length: 2871, dtype: object

In [81]:
X_test[0]

array([  296,    41,   380,     7,    67,   531,     5,     7,   342,
       10452,     2,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [82]:
test_sentences = tokeniser.sequences_to_texts(X_test)

In [83]:
test_sentences[0]

'French officials confirmed a new case in a dead swan .'

In [84]:
y_test[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [86]:
preds[0]

array([[2.7035462e-02, 1.4811838e-01, 3.9743781e-03, ..., 1.3823706e-03,
        5.5929041e-04, 3.1204775e-03],
       [6.3034691e-05, 8.9686364e-05, 3.0238766e-04, ..., 2.3122574e-04,
        4.5133016e-05, 4.0437648e-05],
       [4.7375954e-05, 2.5247733e-05, 4.5294632e-06, ..., 3.8753380e-04,
        9.3793878e-06, 9.6761084e-07],
       ...,
       [1.2897041e-05, 6.5935756e-06, 8.3452164e-08, ..., 9.7516822e-06,
        6.4121002e-07, 4.1073985e-08],
       [1.3884485e-05, 7.1878803e-06, 8.0298776e-08, ..., 9.9454082e-06,
        6.7405853e-07, 3.9778843e-08],
       [1.4700182e-05, 7.6855458e-06, 7.8034113e-08, ..., 1.0111731e-05,
        7.0243840e-07, 3.8859884e-08]], dtype=float32)

In [87]:
pred_data['pred_target_index'][0]

0    13
0     9
0     5
0     5
0     5
     ..
0     5
0    13
0     9
0     5
0     5
Name: pred_target_index, Length: 2871, dtype: int64

In [88]:
pred_data['test_input'][0]

0      296
0      108
0      148
0       47
0       59
     ...  
0     1106
0      179
0      331
0    11961
0      447
Name: test_input, Length: 2871, dtype: int32

In [89]:
pred_data.shape

(63064, 8)

In [92]:
X_test.shape

(2871, 128)

In [93]:
fin_data = pred_data.groupby('id')

In [94]:
fin_data.head()

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,296,296,French,13,13,B-gpe,B-gpe,1
1,41,41,officials,5,5,O,O,1
2,380,380,confirmed,5,5,O,O,1
3,7,7,a,5,5,O,O,1
4,67,67,new,5,5,O,O,1
...,...,...,...,...,...,...,...,...
0,447,447,Last,5,5,O,O,2871
1,75,75,week,5,5,O,O,2871
2,3,3,",",5,5,O,O,2871
3,1334,1334,Zimbabwe,0,0,B-org,B-org,2871


In [95]:
pred_data[pred_data['id']==1]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,296,296,French,13,13,B-gpe,B-gpe,1
1,41,41,officials,5,5,O,O,1
2,380,380,confirmed,5,5,O,O,1
3,7,7,a,5,5,O,O,1
4,67,67,new,5,5,O,O,1
5,531,531,case,5,5,O,O,1
6,5,5,in,5,5,O,O,1
7,7,7,a,5,5,O,O,1
8,342,342,dead,5,5,O,O,1
9,10452,10452,swan,5,5,O,O,1
